In [1]:
import pandas as pd
import numpy as np
import io
from tqdm import tqdm
from itertools import islice
import langid
import re
from concurrent import futures
import swifter
import asyncio
import nltk
# import vaex
from nltk import  pos_tag
from nltk import WordNetLemmatizer
import modin.pandas as pdmd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import multiprocessing as mp


import warnings
warnings.filterwarnings("ignore")

In [2]:
documents = pd.read_pickle('data')

,issue_url,issue_title,body,words_body,words_title,vectors_body,vectors_title
0,"""https://github.com/atais/angular-eonasdan-dat...",manually entered dates issues,i use format 'yyyy-mm-dd' option and bind ng-m...,"[becomes, datetimepicker, value, display, mess...","[date, entered, issue, manually]","[-1.0918881048412594, 0.6091022989154856, -0.0...","[-0.1718999730348316, 0.013864445735690257, 0...."
1,"""https://github.com/conveyal/analysis-ui/issue...",highlight segment on map when editing speed,when editing the speed of a segment i.e. when ...,"[i.e, figure, count, map, edit, either, speed,...","[segment, highlight, map, edit, speed]","[-0.29115673125242214, 0.3148876573334518, -0....","[-0.09172449529209924, 0.6120846510440893, -0...."
2,"""https://github.com/Tat5ato/Phantasmic-Mind/is...",concept art for the otherworld,"in general, the otherworld is craggy and organ...","[color, making, green, general, dungeon, refer...","[otherworld, art, concept]","[-0.6426295987635459, 0.1959413906015991, -0.6...","[-0.2129118350085444, 0.1997058162846672, 0.08..."
3,"""https://github.com/qmlweb/qmlweb/issues/420""",mousearea and touch event in mobile browser,"hello, in the master branch, mousearea doesn't...","[therefore, able, handle, browser, mousearea, ...","[touch, mobile, mousearea, browser, event]","[-0.0860627510084977, 0.08294770021710883, -0....","[-0.18239689955900826, 0.11999946835429123, -0..."
4,"""https://github.com/pybel/pybel/issues/174""","function to drop graph store and edge store, b...",this function should go in the cache manager. ...,"[cache, also, cli, expose, manager, graph, man...","[edge, drop, definition, store, graph, function]","[0.023231257312182667, 0.49292497099851956, -0...","[-0.12271752763802979, 0.42897454135271296, -0..."
...,...,...,...,...,...,...,...
9311,"""https://github.com/nextcloud/serverinfo/issue...",settings button gone,when i deselected all kinds of logging entries...,"[image, kind, button, ie11, nextcloud, setting...","[setting, button]","[-0.30116717000134485, 0.29315197427729217, -0...","[-0.1860621278884357, 0.08401209127442986, 0.1..."
9312,"""https://github.com/WhisperSystems/Signal-Andr...",using without gms,"hello, one of my friends have some technical p...","[privacy, register, phone, google, custom, ver...","[without, use]","[-0.276435802930278, -0.30153380350044034, -0....","[-0.14858783265086034, -0.06445682709821797, -..."
9313,"""https://github.com/automl/auto-sklearn/issues...",no module named 'configspace.io' during import...,i encounter this exception when trying to impo...,"[configspace0.4.3, modulenotfounderror, auto-s...","[module, auto-sklearn, configspace.io, import,...","[-0.29316283410664706, 0.6230892342534959, -0....","[-0.17694951733802847, 0.2904565066593103, -0...."
9314,"""https://github.com/gimli-org/gimli/issues/50""",ship self-testing binaries,"all distributed binaries of pygimli windows, c...","[binary, distributed, conda, pg.test, etc, pac...","[binary, self-testing, ship]","[-0.0914759509305484, 0.4487135361928359, -0.4...","[0.3160170426419626, 0.2812387900777582, -0.13..."


In [3]:
def get_tfidf(type_tfidf):
    dummy_fun = lambda doc: doc
    tfidf = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None)

    if type_tfidf == 'body':
        tfidf.fit(documents.words_body)
    elif type_tfidf == 'title':
        tfidf.fit(documents.words_title)

    return tfidf

In [ ]:
tfidf_title = get_tfidf('title')

In [23]:
def load_vectors(fname, limit):
  fin = io.open(fname, 'r', encoding = 'utf-8', newline = '\n', errors = 'ignore')
  n, d = map(int, fin.readline().split())
  data = {}
  for line in islice(fin, limit):
    tokens = line.rstrip().split(' ')
    data[tokens[0]] = np.array(list(map(float, tokens[1:])))
  return data
  
vecs = load_vectors('crawl-300d-2M.vec', 1_000_000)

KeyboardInterrupt: 

In [24]:
def get_vocab(tfidf):
    dim = 300
    zero = sum(vecs.values()) / len(vecs)

    vocab = np.zeros((len(tfidf.vocabulary_.keys()), dim))
    for key in tfidf.vocabulary_.keys():
        vocab[tfidf.vocabulary_[key]] = vecs.get(key, zero)
    return vocab

vocab_title = get_vocab(tfidf_title)

In [30]:
tfidf_t.shape, vocab_title.shape

((11228,), (11228, 300))

In [35]:
import preprocess
from sklearn.metrics.pairwise import cosine_similarity

words = preprocess.preprocessing_text("python module")
doc = [-0.4462503948404968, 0.28359947183059475, -0.07101933531597554, -0.12367797319164262, 0.03761754315685056, -0.06284692518912936, -0.17654293345032163, -0.6419672914634875, -0.2189299257473043, -0.09615080152198448, 0.5486512024613589, 1.3031293955204672, 0.17161140372588438, -0.2757281769571222, -0.3623743613876279, 0.20602744161792644, -0.3309238236464494, -0.39950838819438167, 0.11132004487500331, -0.32773019295490435, -0.25156560272558065, -0.25736439535604305, -0.06623616218104483, 0.11857001462848595, -0.2930991895563532, -0.16180646256270967, -0.009082833623809962, 0.5290960011517228, 0.01947972915853923, -0.07653202152835224, -0.12607731433463815, 0.22462154862414332, 0.35970346173360496, -0.3568170237335682, 0.16605134053935297, 0.11287234287202422, 0.14168741375168764, -0.19261765204882875, 0.08416413091988872, 0.07369445692231179, 0.4482754729849256, 0.10014234327636556, -0.21043060902973584, 0.20354034940331145, -0.10077453999313676, -0.20736229942040338, 0.07298041982625857, -0.3064768957546244, 0.18786202204816155, -0.49580216443078073, -0.05295030999517184, 0.17855722764178578, 1.2064420822088482, 0.3877644537984423, 0.3154685502472659, -0.21045614507417218, -0.13129983332077175, -0.391135908821944, -0.07385206945108501, -0.099888671411937, 0.33333537997883816, -0.30011960623619466, -0.3843015228888994, -0.6820061818504362, -0.09119323101618936, -0.017315092805138255, 0.42697285615105685, 0.2541135251914139, -0.048077563531251435, -0.6462392169418902, -0.38349317417969925, 0.4452057538105566, -0.2379425406373532, -0.49735126837629906, -0.013426614078556475, 0.3364422496098475, -1.0077815964446022, 0.05172212186623442, -0.16348465888630392, 0.01871289080921157, 1.2528103734317446, 0.04798193653787757, 0.15059129593682524, -0.4847027269279395, 0.07193476006825869, -0.21903518991767465, -0.13695314960225544, 0.0615593102754291, 0.06609550424777422, -0.02041544100031911, -0.29626383861494665, 0.041291388008033066, 0.03114591482532462, 0.3979555864888097, -0.4261085157481374, 0.13915386845219557, -0.05477021161307617, -0.6110104216976547, -0.6338449431488189, -0.45416655704056885, 0.11316430807474495, -0.026164889153596018, -0.19727305592392788, -0.19052601649102902, 0.12688594686137333, -0.22614650677392306, -0.45684071693922274, 0.22449325361234757, -0.14840223789818735, -0.3843277286390108, -0.47039508050665396, 0.0014631893612278477, -0.0748921053715693, 0.23339013895194885, -1.4909594335281258, 0.20242120929554486, 0.4714638201238134, -0.18672538969364372, -0.18561607038547065, -0.1505848252148848, 0.057870648288659336, -0.15994665173491807, -0.023517680759821538, 0.08892363772286989, 0.1751769629932845, -0.150409811795376, -0.348496180387872, 0.07585780085320178, 0.24967165418602089, -0.009511710104114491, 0.6296908324582355, 0.07138699637223753, -0.4367669236502484, 0.29911378787546117, 0.3643560820635339, -0.2104278209174152, -0.015162670516785208, 0.15079801354346944, 0.30498537861525454, -1.335386411520614, -0.37386890481156865, 0.15013922462583756, -0.2488162429209211, -0.27365286656095283, -0.5271307594282592, -0.25030233037631, -0.1755737118997925, -0.6317080278142716, -0.5878440850411626, -0.01830346269923071, 0.26174565030872543, -0.24003699409201074, -0.3241176016529991, -0.11048190335493015, -0.18805498052928518, -0.14811377710607812, 0.20276498481901475, -0.5005430416519498, -0.17254707487795704, 0.09429991285622571, 1.2523169508659908, -0.26673918978967714, 0.21182723582859692, -0.3772596420883174, -0.24826652111336447, -0.3390941288576281, -0.048802822701150037, 0.8675480518114969, 0.06210854216474165, -0.06525993496900077, 0.1233226390394036, 0.39513263565750045, -0.20565351215339125, -0.21725817630855515, -0.8005896704535393, 0.6170187187814357, -0.03678178449013569, 0.1396462333366266, -0.07785183150385915, -0.05785930363383411, -0.1025612440023526, -0.13513916125913505, -0.018026245347045066, -0.009712135537484633, 0.0709463324727992, 0.32424809598112564, -0.008754439767726597, 0.21081171685392835, 0.2531258439109553, -0.3221054647217262, 0.121732285615354, 0.06928343613141019, -0.2493300768839801, 0.3384620678316325, -0.0017840709677125494, -0.5746172479377677, 0.5290753307371547, -0.12596404812584824, 0.31097193258787215, 0.02242918521906639, 0.0675900584912971, 0.1714681046966384, -0.1390388017823463, -0.3443480321237554, -0.356072307984328, 0.1579152702112735, 0.011650304821917805, 0.23708632947818842, 0.5547649463968651, 0.17741685905262417, -0.2718470092021528, 1.6250489102772545, -0.36477386002382645, -0.1987193947011948, 0.08015038002288395, 0.2135513953958716, -0.09158519104510762, 0.23307520950536098, 0.26472182962466245, 0.02749277230212442, -0.4663035058247127, 0.5373248799603298, -0.18568750773339285, -0.10154893779612736, 0.22247926699080944, 0.05095960274908797, 0.41510040237058715, 0.1599233584219622, -0.19914293569377728, 0.34847288464836407, 0.23198907081847236, 0.421914024198894, -0.036946026472186456, -0.0018398909830824807, 0.15132162678161068, -1.2219093018158407, -0.26306915588326096, 0.07627903886611875, 0.572709783580723, -0.4165952720257992, -0.08501229916884309, 0.1647497249927206, 0.46279749937674197, -0.01671249447149237, 0.12775415467040702, -0.2450165868842343, 0.23663496636234002, 0.33012962618823105, 0.2330782688007993, -0.11640414361588251, -0.14902562160725444, 0.3481982866556055, 0.3255774183117795, -0.6730700735507189, 0.13182643039512332, 0.019358819525428887, 0.21186059651797162, -0.14394896688651773, 0.4142822306311954, -0.24999802938457616, 0.0847207835682903, 0.23063627714866572, 0.03785756285561656, 0.2766775973014874, -0.0430646654834772, 0.15973156548375295, 0.03191305632543997, 0.34343874104443717, -0.5087268874196248, -0.2497241710442983, 0.06440047213356549, -0.13087789552650014, 0.3060017500390421, 0.46694218520528963, 0.1504813274315327, 0.20493340808274735, -0.48849240297667923, 0.3547643541721378, 0.11743462763592713, 0.41117328794876923, 0.08644790088500082, -0.22652471695244789, -0.2624379633850216, -0.15074015458927592, -0.388745529618522, -0.42698118090955706, -0.13154374853397136, 0.28012974325323237, 0.15201499269733043, 0.03565916851335873, 0.13012450292200825, 0.14103290215704517, 0.05334818270175863, 0.2059837191904783, 0.03124287578872662, 0.2817439962452036, -0.008326669020692517, -0.5423750229211063, 0.09667096685713972, 0.04356054957040762]
tfidf_t = tfidf_title.transform([words]).toarray().squeeze()

vector_title = tfidf_t.dot(vocab_title)

# np.array(data.vectors_body.tolist()).dot(vocab_body).tolist()
diff_title = cosine_similarity([vector_title], [doc])[0][0]

In [36]:
diff_title

0.6298312876168672

In [22]:
vector_title.shape
np.array(doc).shape

(300,)

In [18]:
diff_title

0.0

# Создание масенького датасета из большого

Нашла я датасет гитхабовских проблем на 2гб

In [4]:
data = pd.read_csv("github_issues.csv")
data

,issue_url,issue_title,body
0,"""https://github.com/zhangyuanwei/node-images/i...",can't load the addon. issue to: https://github...,can't load the addon. issue to: https://github...
1,"""https://github.com/Microsoft/pxt/issues/2543""",hcl accessibility a11yblocking a11ymas mas4.2....,user experience: user who depends on screen re...
2,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1265: issue 1264: issue 1261: issue 1260...,┆attachments: <a href= https:& x2f;& x2f;githu...
3,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1266: issue 1263: issue 1262: issue 1259...,gitlo = github x trello\n---\nthis board is no...
4,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1288: issue 1285: issue 1284: issue 1281...,┆attachments: <a href= https:& x2f;& x2f;githu...
...,...,...,...
5332148,"""https://github.com/bayborodin/ror-full-3/issu...","создать модуль instancecounter, содержащий сле...","методы класса: - instances, который возвращает..."
5332149,"""https://github.com/eclipse/paho.mqtt.java/iss...",at org.eclipse.paho.client.mqttv3.internal.cli...,- bug exists release version 1.1.0 master bran...
5332150,"""https://github.com/rzwitserloot/lombok/issues...",java.lang.linkageerror: loader constraint viol...,java.lang.linkageerror: loader constraint viol...
5332151,"""https://github.com/Gizra/productivity/issues/...",node : pdoexception: sqlstate 40001 : serializ...,view details in rollbar: https://rollbar.com/b...


In [9]:
data.describe()

,issue_url,issue_title,body
count,5332153,5332153,5332153
unique,5315477,4862563,5041808
top,"""https://github.com/cmty-test/cmty-repository-...",first from flow in uk south,first from flow in uk south
freq,297,90133,90133


In [10]:
slice_data = data.sample(frac=1, random_state=42)[:500_000]
slice_data = slice_data.reset_index(drop=True)

In [11]:
slice_data.to_csv("github_issues_slice.csv", index=False)

## Проведу мини-анализ по ускорению обработки данных

Для начала мне нужно выделить из исходного датасета только текст на английском языке (мне так хочется). Для этой задачи уже можно попробовать ускорять

In [33]:
def is_en(text):
    return text if langid.classify(text)[0] == 'en' else pd.NA

In [5]:
def is_en_md(text):
    return text if langid.classify(text)[0] == 'en' else pdmd.NA

In [34]:
def is_en_bool(text):
    return langid.classify(text)[0] == 'en'

### read_csv

In [54]:
# %%timeit

slice_data = pd.read_csv("github_issues_slice.csv")[:1000]

In [55]:
# %%timeit

slice_data_md = pdmd.read_csv("github_issues_slice.csv")[:1000]

### Apply by pandas and map

In [21]:
%%timeit

to_save = slice_data[slice_data.body.map(is_en_bool)]
# slice_data = slice_data[to_save.index]
len(to_save)

9.14 s ± 430 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit

res = slice_data.body.apply(is_en)
res.dropna()

8.9 s ± 181 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

res = slice_data_md.body.apply(is_en_md)
res.dropna()

7.18 s ± 139 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### future ThreadPoolExecutor

In [57]:
# %%timeit

with futures.ThreadPoolExecutor() as executor:
    res = pd.Series(executor.map(is_en, slice_data.body))
    res.dropna()

In [8]:
# %%timeit

with futures.ThreadPoolExecutor() as executor:
    res = pdmd.Series(executor.map(is_en_md, slice_data_md.body))
    res.dropna()

### future ProcessPoolExecutor

In [49]:
%%writefile ProcessPoolExecutor_pd.py

import pandas as pd
import langid
import time
from concurrent import futures
import numpy as np

def is_en(text):
    return langid.classify(text)[0] == 'en'

def process_series(series: pd.Series):
    return series.map(is_en)

def main(df):
    n = 4
    parts = np.array_split(df.body, n)

    print("START TIME")
    start = time.perf_counter()

    with futures.ProcessPoolExecutor() as executor:
        res = executor.map(process_series, parts)

    print(f"END TIME: {time.perf_counter() - start}")
    print(res)


if __name__ == '__main__':
    df = pd.read_csv("github_issues_slice.csv")[:1000]

    main(df)

Overwriting ProcessPoolExecutor_pd.py


In [50]:
!python ProcessPoolExecutor_pd.py

START TIME
END TIME: 10.9749333
<generator object _chain_from_iterable_of_lists at 0x000001E022B25740>


In [27]:
%%writefile ProcessPoolExecutor_pdmd.py

import pandas as pd
import modin.pandas as pdmd
import langid
import time
from concurrent import futures


def is_en(text):
    return text if langid.classify(text)[0] == 'en' else pd.NA

def main(df):
    print("START TIME")
    start = time.perf_counter()

    with futures.ProcessPoolExecutor() as executor:
        res = pdmd.Series(executor.map(is_en, df.body))

    print(len(res.dropna()))
    print(f"END TIME: {time.perf_counter() - start}")

if __name__ == '__main__':
    df = pdmd.read_csv("github_issues_slice.csv")[:1000]

    main(df)

Writing ProcessPoolExecutor_pdmd.py


In [28]:
!python ProcessPoolExecutor_pdmd.py

### swifter

In [21]:
%%timeit

res = slice_data.body.swifter.apply(is_en)
res.dropna()

Pandas Apply: 100%|██████████| 1000/1000 [00:08<00:00, 111.75it/s]

8.88 s ± 885 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Process

In [51]:
%%writefile process_mp.py

import pandas as pd
import langid
import time
from concurrent import futures
import numpy as np
import multiprocessing as mp


def is_en_bool(text):
    return langid.classify(text)[0] == 'en'

def process_series(series: pd.Series):
    return series.map(is_en_bool)

def main_n_process(parts):
    processes = [
        mp.Process(target=process_series, args=(part,))
        for part in parts
    ]
    
    for process in processes:
        process.start()

    print("START TIME")
    start = time.perf_counter()
    
    for process in processes:
        process.join()

    print(f"END TIME: {time.perf_counter() - start}")
    return process

if __name__ == '__main__':
    slice_data = pd.read_csv("github_issues_slice.csv")[:1000]
    n = 4
    parts = np.array_split(slice_data.body, n)
    result = main_n_process(parts)
    print(result)

Overwriting process_mp.py


In [52]:
!python process_mp.py

START TIME
END TIME: 7.654426100000002
<Process name='Process-4' pid=10088 parent=13892 stopped exitcode=0>


### vaex

In [5]:
dv = vaex.from_csv("github_issues_slice.csv", convert=True, chunk_size=1_000_000)

In [6]:
dv = vaex.open('github_issues_slice.csv.hdf5')[:1000]

In [7]:
def is_en(text):
    return np.where(langid.classify(text)[0] == 'en', text, np.nan)
    # return text if langid.classify(text)[0] == 'en' else np.nan

In [8]:
dv.body = dv.body.apply(is_en)

In [9]:
%%time
len(dv.dropna())

Wall time: 27 s


1000

# Result

In [1]:
def is_en(text):
    return text if langid.classify(text)[0] == 'en' else pd.NA

In [4]:
data = pd.read_csv("github_issues_slice.csv")[:50_000]

In [ ]:
with futures.ThreadPoolExecutor() as executor:
    data.body = pd.Series(executor.map(is_en, data.body))

In [ ]:
len(data[data.body.isnull()])

6577

In [ ]:
data = data.dropna()

In [ ]:
data

,issue_url,issue_title,body
0,"""https://github.com/atais/angular-eonasdan-dat...",manually entered dates issues,i use format 'yyyy-mm-dd' option and bind ng-m...
1,"""https://github.com/conveyal/analysis-ui/issue...",highlight segment on map when editing speed,when editing the speed of a segment i.e. when ...
2,"""https://github.com/Tat5ato/Phantasmic-Mind/is...",concept art for the otherworld,"in general, the otherworld is craggy and organ..."
3,"""https://github.com/qmlweb/qmlweb/issues/420""",mousearea and touch event in mobile browser,"hello, in the master branch, mousearea doesn't..."
4,"""https://github.com/pybel/pybel/issues/174""","function to drop graph store and edge store, b...",this function should go in the cache manager. ...
...,...,...,...
99995,"""https://github.com/aurelia/dialog/issues/288""",the change dialog-renderer: let click events b...,i'm submitting a bug report operating system: ...
99996,"""https://github.com/joyent/triton/issues/231""",is this project dead?,in looking at some of the outstanding issues w...
99997,"""https://github.com/ARTdrakon/Squad/issues/4""",nicknames with non utf-8 symbols sometimes inv...,player nickname with non utf-8 symbols sometim...
99998,"""https://github.com/npm/npm/issues/19378""",run andoid emulator,c:\users\user>emulator @nexus_5x_api_27 :error...


In [ ]:
data = data.reset_index(drop=True)

# Обработка данных

In [ ]:
data.body[0]

"i use format 'yyyy-mm-dd' option and bind ng-model= formdata.date to my field. now if i enter '2015-03-05aaaaa', the date is displayed as '2015-03-05', but my $scope.formdata.date becomes '2015-03-05aaaaa'. so even if i write error message to user that date is invalid, in input field it looks like totally valid value '2015-03-05'. how do i stop datetimepicker from automatically correcting my dates? this misleads users."

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Sentence segmentation

In [ ]:
def sentence_segmentation(text: str) -> list:
    if re.match(r'[\.!\?;]', text[-1]):
        text = text[:-1]
    return re.split(r'[\.!\?;]\s', text.lower())

## Tokenization

In [ ]:
def tokenization(sentences: list) -> list:
    return [re.split(r'[,:(\s\-)]*\s', s) for s in sentences]

## Lemmatization

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wn.ADJ,
        'V': wn.VERB,
        'N': wn.NOUN,
        'R': wn.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wn.NOUN

In [ ]:
def lemmatization(sentences: list) -> list:
    sentences_tag  = [pos_tag(s) for s in sentences] # получаем теги слов каждого предложения
    
    lemmatizer = WordNetLemmatizer()
    lemm_sentences = []
    for sent in sentences_tag:
        pos_tagged = [(word, get_wordnet_pos(tag)) for word, tag in sent]
        lemm_sentences.append([lemmatizer.lemmatize(word, tag) for word, tag in pos_tagged])

    return lemm_sentences

## Delete stopwords

In [ ]:
stop_words = set(stopwords.words('english')).union({'', ' '})

In [ ]:
def del_stopwords(sentences: list) -> list:
    upd_sentences = []
    re_sub = lambda x: re.sub(r"[\+=\t\r\n,;:\*'\"]+","", x)
    union_sentences = lambda x: list(set().union(*x))

    for sent in sentences:
        upd_sentences.append([
            re_sub(word) for word in sent if re_sub(word) not in stop_words and len(word) not in [1, 2]
        ])
    
    return union_sentences(upd_sentences)

## All

In [ ]:
def preprocessing_text(text: str) -> list:
    return del_stopwords(lemmatization(tokenization(sentence_segmentation(text))))

In [ ]:
with futures.ThreadPoolExecutor() as executor:
    data['words_body'] = pd.Series(executor.map(preprocessing_text, data.body))

In [ ]:
with futures.ThreadPoolExecutor() as executor:
    data['words_title'] = pd.Series(executor.map(preprocessing_text, data.issue_title))

## Vectorization

In [ ]:
def load_vectors(fname, limit):
  fin = io.open(fname, 'r', encoding = 'utf-8', newline = '\n', errors = 'ignore')
  n, d = map(int, fin.readline().split())
  data = {}
  for line in tqdm(islice(fin, limit), total = limit):
    tokens = line.rstrip().split(' ')
    data[tokens[0]] = np.array(list(map(float, tokens[1:])))
  return data

vecs = load_vectors('crawl-300d-2M.vec', 1_000_000)  

100%|██████████| 1000000/1000000 [03:19<00:00, 5005.40it/s]


In [ ]:
def dummy_fun(doc):
    return doc

In [ ]:
tfidf_body = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None)
tfidf_body.fit(data.words_body)

tfidf_title = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None)
tfidf_title.fit(data.words_title)

TfidfVectorizer(preprocessor=<function dummy_fun at 0x000002595E576E50>,
                token_pattern=None,
                tokenizer=<function dummy_fun at 0x000002595E576E50>)

In [ ]:
def vectorization_body(words: list):
    return tfidf_body.transform([words]).toarray().squeeze()

def vectorization_title(words: list):
    return tfidf_title.transform([words]).toarray().squeeze()

In [ ]:
with futures.ThreadPoolExecutor() as executor:
    data['vectors_body'] = pd.Series(executor.map(vectorization_body, data.words_body))

MemoryError: bad allocation

In [ ]:
with futures.ThreadPoolExecutor() as executor:
    data['vectors_title'] = pd.Series(executor.map(vectorization_title, data.words_title))

In [ ]:
dim = 300
zero = sum(vecs.values()) / len(vecs)

In [ ]:
vocab_body = np.zeros((len(tfidf_body.vocabulary_.keys()), dim))
for key in tqdm(tfidf_body.vocabulary_.keys()):
  vocab_body[tfidf_body.vocabulary_[key]] = vecs.get(key, zero)

100%|██████████| 56719/56719 [00:01<00:00, 47862.57it/s]


In [ ]:
vocab_title = np.zeros((len(tfidf_title.vocabulary_.keys()), dim))
for key in tqdm(tfidf_title.vocabulary_.keys()):
  vocab_title[tfidf_title.vocabulary_[key]] = vecs.get(key, zero)

100%|██████████| 11228/11228 [00:00<00:00, 92026.15it/s]


In [ ]:
data['vectors_body'] = np.array(data.vectors_body.tolist()).dot(vocab_body).tolist()
data['vectors_title'] = np.array(data.vectors_title.tolist()).dot(vocab_title).tolist()

In [ ]:
data.to_pickle("data")

In [ ]:
import pickle

def save_pickle(obj, file):
    with open(file, 'wb') as f:
        pickle.dump(obj, f)

In [ ]:
save_pickle(tfidf_title, "tfidf_title.pickle")
save_pickle(tfidf_body, "tfidf_body.pickle")